# 3dlss

## Running 3dlss via afni

Like for 3dDeconvolve, we need stimulus timing files, nuisance regressors, and censored timeploints. Run the 3dDeconvolve command in the preprocessing program to generate these.

3dlss only takes one input so we need to first concatenate the runs using 3dTproject. This willl generate a errts.nii.gz file  


```
stimuli=(dcb fcb dpb fpb dcr fcr dpr fpr)
get_stimuli() {
    IM_index=$1
    shift=$2
    stimuli_index=$(((IM_index + shift) % 8))
    stimulus=${stimuli[$stimuli_index]}
    echo $stimulus
    return 0
}
subjects=(10009 10010)
model='TENT(6, 20.4, 9)'

for subject in "${subjects[@]}"; do
    echo "Starting 3dDeconvolve on $subject"
    outputpath="/data/backed_up/shared/ThalHi_MRI_2020/3dDeconvolve/sub-${subject}"
    data="/data/backed_up/shared/ThalHi_MRI_2020/fmriprep/sub-${subject}/func"
    cd $outputpath

    3dmask_tool -input $(find /data/backed_up/shared/ThalHi_MRI_2020/fmriprep/sub-${subject}/func/*mask.nii.gz) -overwrite
    
    # run 3dDeconvolve in two steps
    # because 3dLSS will not take more than one "input", we need a clever way to concatenate the runs.
    # So we first run a nuisance regression (3dTproject) and use the residuals for model fitting. Here the residuals are "denoised" data
    3dTproject -input $(ls ${data}/sub-${subject}_task-ThalHi_run-*space-MNI152NLin2009cAsym_desc-preproc_bold*.nii.gz | sort -V) \
    -mask combined_mask+tlrc.BRIK \
    -polort 3 \
    -ort nuisance.1D \
    -prefix errts.nii.gz \
    -overwrite

```

__We will then loop through each stimulus and run the 3dDeconvolve to generate a design matrix. Because the errts.nii.gz file from 3dTproject is a concatenation of all 8 runs. We need to manually setup a -concat to tell 3dDeconvolve where are the run breaks, otherwise "local times" won't work properly. Also, note that if you have a subject with different number of runs or different run length, you will have to set this up differently.__
  
__As for the stim_times, here we are going to pull out single trial betas for the "dcb" condition, while putting all other conditions in the model. We will have to do iterate through this 8 times, each time using a different condtion for the stim_times_IM option, while leaving the others in the stim_times column.__

```
  for i in {0..7}; do {
        3dDeconvolve -input errts.nii.gz \
        -concat '1D: 0 216 432 648 864 1080 1296 1512' \
        -mask combined_mask+tlrc.BRIK \
        -censor censor.1D \
        -x1D $(get_stimuli $i 0).xmat.1D \
        -local_times \
        -stim_times_subtract 0.9 \
        -num_stimts 8 \
        -stim_times_IM  1 $(get_stimuli $i 0).1D.txt $model -stim_label 1 $(get_stimuli $i 0)] \
        -stim_times  2 $(get_stimuli $i 1).1D.txt $model -stim_label 2 $(get_stimuli $i 1) \
        -stim_times  3 $(get_stimuli $i 2).1D.txt $model -stim_label 3 $(get_stimuli $i 2) \
        -stim_times  4 $(get_stimuli $i 3).1D.txt $model -stim_label 4 $(get_stimuli $i 3) \
        -stim_times  5 $(get_stimuli $i 4).1D.txt $model -stim_label 5 $(get_stimuli $i 4) \
        -stim_times  6 $(get_stimuli $i 5).1D.txt $model -stim_label 6 $(get_stimuli $i 5) \
        -stim_times  7 $(get_stimuli $i 6).1D.txt $model -stim_label 7 $(get_stimuli $i 6) \
        -stim_times  8 $(get_stimuli $i 7).1D.txt $model -stim_label 8 $(get_stimuli $i 7) \
        -x1D_stop \
        -allzero_OK \
        -jobs 2
```


__Now we will run 3dlss using the concatenated runs from 3dTproject and the .xmat.1D design matrix from 3dDeconvolve. Again, remember we will have to do this 8 diff times, each time using a different .xmat.1D file from the previous step.__
```
        # step 3 3dLSS and run it.
        3dLSS -input errts.nii.gz \
        -matrix $(get_stimuli $i 0).xmat.1D \
        -prefix $(get_stimuli $i 0).LSS \
        -overwrite
        } &
    done
    wait
done
```

__The output will be a {stimulus}.LSS file in the subject's 3dDeconvolve directory.__

## Using output with LSS class

Now that you have your LSS data you can convert it to a data object for easier manipulation.
From the decoding.py file import the SubjectLssTentData class and instantiate by passing the subject deconvolve directory and list of cues you are working with.

In [ ]:
from thalpy.analysis.lss import SubjectLssTent

CUES = ["dcb", "fcb", "dpb", "fpb", "dcr", "fcr", "dpr", "fpr"]
deconvolve_sub_dir = "/data/backed_up/shared/ThalHi_MRI_2020/3dDeconvolve/sub-10001"
lss_tent = SubjectLssTent(deconvolve_sub_dir, CUES) # tent length and path to save object at are optional arguments

Instantiating the object automatically converts the lss files into a numpy array in the shape of [voxel_x, voxel_y, voxel_z, tent_length * num_trials] where the last dimension is the tent length multiplied by the number of trials. For example, with a tent length of 9, the first trial and first tent would be index 0, the 1st trial and 2nd tent would be index 1 ... and the 9th tent would be 8. Then, the second trial would then start at index 9 and continue the same pattern. So it goes.

Additionally, the object creates an average of the tents for each trial, which is stored in another numpy array in the shape of [voxel_x, voxel_y, voxel_z, num_trials].

Finally, the object removes all nan trials (censored) from both matrices and saves (via pickle) the object.

In [ ]:
# You can access the trials with all the tents via the .data property
print(lss_tent.data)
print(lss_tent.data.shape)

# You can access the averaged tents per trial via the .avg_data property
print(lss_tent.avg_data)
print(lss_tent.avg_data.shape)


You can also make changes to the object and then save them using the .save() function. The path is optional

In [ ]:
lss_tent.save("new_lss_data.p")

If you have previously created your object and saved it. You can load it by using the static load function and passing the filepath. This will load the object and you can use the same functions and access the same properties as previously discussed.

In [ ]:
lss_tent = SubjectLssTentData.load("path_to_lss/lss.p")